In [2]:
import sys

if not 'texar_repo' in sys.path:
  sys.path += ['texar_repo']
import tensorflow as tf
import texar as tx
import numpy as np
from config import *
from model import *

import os




def _eval_epoch(sess, epoch, mode):

    diff,references, hypotheses = [], [], []

    if mode == 'test':
        iterator.restart_dataset(sess, 'test')
        bsize = test_batch_size
        fetches = {
        'inferred_ids': inferred_ids,
        }
        bno=0

    with open('generation.txt','w') as f:
        while True:

            #print("Temp",temp)
            try:
                print("Batch",bno)
                feed_dict = {
                    iterator.handle: iterator.get_handle(sess, 'test'),
                    tx.global_mode(): tf.estimator.ModeKeys.PREDICT,
                }
                op = sess.run([batch],feed_dict)
                
                feed_dict = {
                   src_input_ids:op[0]['src_input_ids'],
                   src_segment_ids : op[0]['src_segment_ids'],
                   tx.global_mode(): tf.estimator.ModeKeys.PREDICT
                }
                fetches_ = sess.run(fetches, feed_dict=feed_dict)
                labels = op[0]['tgt_labels']
                diffs = op[0]['src_input_ids']
                hypotheses.extend(h.tolist() for h in fetches_['inferred_ids'])
                references.extend(r.tolist() for r in labels)
                diff.extend(d.tolist() for d in diffs)
                
                bno = bno+1

            except tf.errors.OutOfRangeError:
                break
        
        hypotheses = utils.list_strip_eos(hypotheses, eos_token_id)
        references = utils.list_strip_eos(references, eos_token_id)
        diff = utils.list_strip_eos(diff, eos_token_id)

        for example in range(len(references)):
            hwords = tokenizer.convert_ids_to_tokens(hypotheses[example])
            rwords = tokenizer.convert_ids_to_tokens(references[example])
            dwords = tokenizer.convert_ids_to_tokens(diff[example])

            hwords = tx.utils.str_join(hwords).replace(" ##","")
            rwords = tx.utils.str_join(rwords).replace(" ##","")
            dwords = tx.utils.str_join(dwords).replace(" ##","").replace("[CLS]", "")
            f.write('diff: ')
            f.write(dwords)
            f.write('\ngenerated: ')
            f.write(hwords)
            f.write('\nreal ')
            f.write(rwords)
            f.write('\n')
        

tx.utils.maybe_create_dir(model_dir)
logging_file= os.path.join(model_dir,"logging.txt")
logger = utils.get_logger(logging_file)


with tf.device('/gpu:0'):
    with tf.Session() as sess: #config=config
        gpu_options = tf.GPUOptions(allow_growth=True)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))   
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        sess.run(tf.tables_initializer())

        smry_writer = tf.summary.FileWriter(model_dir, graph=sess.graph)

        if run_mode == 'train_and_evaluate':
            logger.info('Begin running with train_and_evaluate mode')

            if tf.train.latest_checkpoint(model_dir) is not None:
                logger.info('Restore latest checkpoint in %s' % model_dir)
                saver.restore(sess, tf.train.latest_checkpoint(model_dir))

            iterator.initialize_dataset(sess)

            #iterator.restart_dataset(sess, 'test')
            step = _eval_epoch(sess, 0, 'test')



        else:
            raise ValueError('Unknown mode: {}'.format(run_mode))


INFO:tensorflow:Restoring parameters from ./models/model_380000.ckpt
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93


In [5]:
from collections import Counter
gen_list = []
ref_list = []
equ_list = []
gen_len = 0
ref_len = 0
first_word_equal = 0
sentence_equal = 0
with open('generation.txt') as f:
    lines = f.readlines()
    print(len(lines))
    for line in lines:
        gen, ref = line.strip().split(' --- ')
        gen_len +=  len(gen.split(' ')) 
        ref_len +=  len(ref.split(' ')) 
        gen_list.append(gen.split(' ')[0])
        ref_list.append(ref.split(' ')[0])
        if gen.split(' ')[0] == ref.split(' ')[0]:
            first_word_equal += 1
        
        if gen == ref:
            sentence_equal += 1
            equ_list.append(ref)
    
    print('first_word_equal', first_word_equal/len(lines))
    print('sentence_equal', sentence_equal/len(lines))
    print('gen_len', gen_len/len(lines))   
    print('ref_len', ref_len/len(lines))
    print(Counter(gen_list))
    print(Counter(ref_list))
    print(Counter(equ_list))

2976
first_word_equal 0.4845430107526882
sentence_equal 0.259744623655914
gen_len 6.815524193548387
ref_len 7.661962365591398
Counter({'ignore': 421, 'add': 354, 'fix': 248, 'updated': 223, 'added': 212, 'remove': 189, 'update': 128, 'fixed': 117, 'lps': 96, 'bump': 61, 'removed': 42, 'prepare': 40, 'updating': 36, 'make': 32, 'adding': 27, 'fixing': 27, 'delete': 23, 'setting': 22, 'changed': 22, 'revert': 22, 'don': 20, 'upgraded': 19, 'use': 19, 'set': 16, 'upgrade': 16, 'create': 15, 'move': 15, 'bumped': 14, 'include': 14, 'rename': 14, 'lrqa': 13, 'do': 12, 'fixes': 10, 'disable': 10, 'push': 10, 'avoid': 9, 'renamed': 9, 'suppress': 8, 'reset': 8, 'created': 7, 'modify': 7, 'minecraftforge': 7, 'change': 7, 'updates': 7, 'prevent': 7, 'sonar': 7, 'specify': 6, 'removing': 6, 'built': 6, 'replace': 6, 'exclude': 6, 'smack': 5, 'run': 5, 'correct': 5, 'moved': 5, 'bumping': 5, 'missing': 5, 'enable': 5, 'corrected': 5, 'improve': 4, 'changing': 4, 'increased': 4, 'deleted': 4, 'gh